In [1]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

perimeters.head()

,geometry
id,
hardbruecke,"POLYGON ((2681783.594 1250389.068, 2682538.818..."
meister_zurich,"POLYGON ((2692470.353 1257302.072, 2692744.903..."
waltikon,"POLYGON ((2688840.134 1243821.068, 2689177.359..."
zurich-birmensdorf,"POLYGON ((2679899.582 1247244.141, 2680185.844..."
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."


In [3]:
regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich'
    ])
]

regions

,geometry
id,
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."
ebc_zrh_v01_ex_zurich,"MULTIPOLYGON (((2671482.755 1238555.241, 26714..."


In [4]:
accessibility = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility_before_after.gpkg')
)

accessibility = gpd.sjoin(
    accessibility,
    regions,
    how='left',
    predicate='within'
).rename(columns={'index_right': 'region'})

accessibility

,accessibility_before,accessibility_cycling_before,accessibility_foot_before,accessibility_private_cars_before,accessibility_transit_before,age,sex,maritalstatus,residencepermit,residentpermit,...,accessibility_cycling_after,accessibility_foot_after,accessibility_transit_after,accessibility_after-before,accessibility_private_cars_after-before,accessibility_cycling_after-before,accessibility_foot_after-before,accessibility_transit_after-before,geometry,region
0,3390.062014,41.506565,9.136406,118.451034,50.638741,38,1,1,301,3,...,59.394663,9.136406,50.638741,-7.456294,-10.782573,17.888098,0.0,0.000000e+00,POINT (2686663.001 1244743.001),ebc_zrh_v01_ex_zurich
1,3413.682104,41.506565,9.136406,118.451034,50.638741,48,2,2,-2,-2,...,59.394663,9.136406,50.638741,-43.512504,-10.782573,17.888098,0.0,0.000000e+00,POINT (2686744.001 1244666.001),ebc_zrh_v01_ex_zurich
2,3430.021641,41.506565,9.136406,118.451034,50.638741,56,1,2,201,2,...,59.394663,9.136406,50.638741,-66.617574,-10.782573,17.888098,0.0,0.000000e+00,POINT (2686703.001 1244729.001),ebc_zrh_v01_ex_zurich
3,3161.453374,38.312947,6.783620,106.949275,39.652278,13,1,1,-2,-2,...,40.334846,6.783620,39.652136,27.576986,0.216207,2.021899,0.0,-1.420971e-04,POINT (2697032.001 1251290.001),ebc_zrh_v01_ex_zurich
4,3166.148945,38.312947,6.783620,106.949275,39.652278,20,2,1,-2,-2,...,40.334846,6.783620,39.652136,27.539145,0.216207,2.021899,0.0,-1.420971e-04,POINT (2696996.001 1251309.001),ebc_zrh_v01_ex_zurich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40690,3398.723625,25.506198,7.699980,99.689766,26.034215,64,2,4,-2,-2,...,26.479891,7.699980,26.034215,3.545620,0.386215,0.973693,0.0,0.000000e+00,POINT (2697206.001 1227810.001),ebc_zrh_v01_ex_zurich
40691,3406.052517,43.440157,8.559276,122.055586,54.701081,34,1,2,-2,-2,...,56.415444,8.559276,54.701081,194.295161,-0.092487,12.975286,0.0,0.000000e+00,POINT (2682877.001 1244282.001),zurich
40692,3117.625715,19.860438,6.163865,82.092081,20.198374,71,2,2,-2,-2,...,21.302140,6.163865,20.198374,-6.804978,0.168224,1.441702,0.0,3.552714e-15,POINT (2699394.001 1227236.001),ebc_zrh_v01_ex_zurich
40693,3064.076906,19.860438,6.163865,82.092081,20.198374,52,2,2,-2,-2,...,21.302140,6.163865,20.198374,-6.947443,0.168224,1.441702,0.0,3.552714e-15,POINT (2699356.001 1227209.001),ebc_zrh_v01_ex_zurich


In [5]:
# temporary fix for missing scaling
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    for state in ['before', 'after', 'after-before']:
        accessibility[f'accessibility_{mode}_{state}'] = accessibility[f'accessibility_{mode}_{state}'] / 0.02

Add group labels

In [6]:
accessibility['x'] = 'all'
accessibility['y'] = 'all'

accessibility['age_group'] = accessibility.apply(
    lambda row: 'age<=25' if row['age'] <= 25 else 'age>=65' if row['age'] >= 65 else 'age:other',
    axis=1
)

accessibility['nationality'] = accessibility.apply(
    lambda row: 'nationality:swiss' if row['residencepermit'] <= -2 else 'nationality:other',
    axis=1
)

accessibility['maritalstatus_type'] = accessibility.apply(
    lambda row: 'maritalstatus:married' if row['maritalstatus'] == 2 else 'maritalstatus:other',
    axis=1
)

accessibility['sex_type'] = accessibility.apply(
    lambda row: 'sex:male' if row['sex'] == 1 else 'sex:female' if row['sex'] == 2 else 'sex:other',
    axis=1
)

stats_values = ['median']

fields = {
    'x': ['count'],
    f'accessibility_{MODE_PRIVATE_CARS}_before': stats_values,
    f'accessibility_{MODE_TRANSIT}_before': stats_values,
    f'accessibility_{MODE_CYCLING}_before': stats_values,
    f'accessibility_{MODE_FOOT}_before': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_after': stats_values,
    f'accessibility_{MODE_TRANSIT}_after': stats_values,
    f'accessibility_{MODE_CYCLING}_after': stats_values,
    f'accessibility_{MODE_FOOT}_after': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_after-before': stats_values,
    f'accessibility_{MODE_TRANSIT}_after-before': stats_values,
    f'accessibility_{MODE_CYCLING}_after-before': stats_values,
    f'accessibility_{MODE_FOOT}_after-before': stats_values,
    f'accessibility_before': stats_values,
    f'accessibility_after': stats_values,
    f'accessibility_after-before': stats_values,
}

accessibility_stats = {
    'region_and_age': accessibility.groupby(['region', 'age_group']).agg(fields),
    'region_and_nationality': accessibility.groupby(['region', 'nationality']).agg(fields),
    'region_and_sex': accessibility.groupby(['region', 'sex_type']).agg(fields),
    'all': accessibility.groupby(['x', 'y']).agg(fields)
}

res = pd.concat(accessibility_stats.values())
res = res.iloc[::-1]

for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    res[f'accessibility_{mode}_after-before'] = (
        res[f'accessibility_{mode}_after-before'] / res[f'accessibility_{mode}_before']
    )
    
res[f'accessibility_after-before'] = (
        res[f'accessibility_after-before'] / res[f'accessibility_before']
    ) 

res.to_excel(
    os.path.join(outputs_path, 'accessibility_results.xlsx'),
)

res

x  \
                                         count   
all                   all                40695   
zurich                sex:male            5473   
                      sex:female          5358   
ebc_zrh_v01_ex_zurich sex:male           14910   
                      sex:female         14849   
zurich                nationality:swiss   7320   
                      nationality:other   3511   
ebc_zrh_v01_ex_zurich nationality:swiss  21876   
                      nationality:other   7883   
zurich                age>=65             1540   
                      age<=25             2745   
                      age:other           6546   
ebc_zrh_v01_ex_zurich age>=65             5209   
                      age<=25             8120   
                      age:other          16430   

                                        accessibility_private_cars_before  \
                                                                   median   
all                   all                                     5446.006962   
zurich                sex:male                                6313.024798   
                      sex:female                              6300.749327   
ebc_zrh_v01_ex_zurich sex:male                                5223.477992   
                      sex:female                              5205.602561   
zurich                nationality:swiss                       6287.436693   
                      nationality:other                       6349.756643   
ebc_zrh_v01_ex_zurich nationality:swiss                       5177.853365   
                      nationality:other                       5325.365444   
zurich                age>=65                                 6243.775991   
                      age<=25                                 6294.575567   
                      age:other                               6330.807718   
ebc_zrh_v01_ex_zurich age>=65                                 5186.818235   
                      age<=25                                 5208.876232   
                      age:other                               5224.593937   

                                        accessibility_transit_before  \
                                                              median   
all                   all                                2136.698450   
zurich                sex:male                           2827.000785   
                      sex:female                         2821.998287   
ebc_zrh_v01_ex_zurich sex:male                           2017.344443   
                      sex:female                         2014.828224   
zurich                nationality:swiss                  2820.425088   
                      nationality:other                  2837.579931   
ebc_zrh_v01_ex_zurich nationality:swiss                  2002.555912   
                      nationality:other                  2059.875844   
zurich                age>=65                            2785.426574   
                      age<=25                            2800.504040   
                      age:other                          2843.171877   
ebc_zrh_v01_ex_zurich age>=65                            2015.490927   
                      age<=25                            2016.244666   
                      age:other                          2015.907277   

                                        accessibility_cycling_before  \
                                                              median   
all                   all                                1911.168900   
zurich                sex:male                           2487.320506   
                      sex:female                         2467.302379   
ebc_zrh_v01_ex_zurich sex:male                           1732.545145   
                      sex:female                         1717.726369   
zurich                nationality:swiss                  2475.668450   
                      nationality:other                  2487.683681   
ebc_zrh_v01

In [7]:
res.sort_index().transpose().to_excel(
    os.path.join(outputs_path, 'accessibility_results_transposed.xlsx'),
)

res.sort_index().transpose()

all  \
                                                         all   
x                                       count   40695.000000   
accessibility_private_cars_before       median   5446.006962   
accessibility_transit_before            median   2136.698450   
accessibility_cycling_before            median   1911.168900   
accessibility_foot_before               median    379.514402   
accessibility_private_cars_after        median   5385.257568   
accessibility_transit_after             median   2136.698450   
accessibility_cycling_after             median   2054.448606   
accessibility_foot_after                median    379.514402   
accessibility_private_cars_after-before median     -0.005147   
accessibility_transit_after-before      median      0.000000   
accessibility_cycling_after-before      median      0.075339   
accessibility_foot_after-before         median      0.000000   
accessibility_before                    median   3374.290561   
accessibility_after                     median   3385.766212   
accessibility_after-before              median      0.002993   

                                               ebc_zrh_v01_ex_zurich  \
                                                           age:other   
x                                       count           16430.000000   
accessibility_private_cars_before       median           5224.593937   
accessibility_transit_before            median           2015.907277   
accessibility_cycling_before            median           1731.035888   
accessibility_foot_before               median            358.294760   
accessibility_private_cars_after        median           5202.535629   
accessibility_transit_after             median           2015.930979   
accessibility_cycling_after             median           1841.924763   
accessibility_foot_after                median            358.294760   
accessibility_private_cars_after-before median             -0.003620   
accessibility_transit_after-before      median              0.000000   
accessibility_cycling_after-before      median              0.063573   
accessibility_foot_after-before         median              0.000000   
accessibility_before                    median           3280.392341   
accessibility_after                     median           3288.648305   
accessibility_after-before              median              0.001518   

                                                                          \
                                                    age<=25      age>=65   
x                                       count   8120.000000  5209.000000   
accessibility_private_cars_before       median  5208.876232  5186.818235   
accessibility_transit_before            median  2016.244666  2015.490927   
accessibility_cycling_before            median  1722.285273  1710.588170   
accessibility_foot_before               median   357.963850   355.774964   
accessibility_private_cars_after        median  5184.385884  5158.847957   
accessibility_transit_after             median  2016.217968  2015.490927   
accessibility_cycling_after             median  1825.036159  1809.385846   
accessibility_foot_after                median   357.963850   355.784378   
accessibility_private_cars_after-before median    -0.003499    -0.003377   
accessibility_transit_after-before      median     0.000000     0.000000   
accessibility_cycling_after-before      median     0.063182     0.062724   
accessibility_foot_after-before         median     0.000000     0.000000   
accessibility_before                    median  3221.137686  3322.907640   
accessibility_after                     median  3233.804488  3326.417802   
accessibility_after-before              median     0.002721     0.001163   

                                                                  \
                                               nationality:other   
x                                       count        7883.000000   
accessibility_private_cars_before 